In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import scipy
from tqdm import tqdm
import copy
import scipy.signal

In [2]:
CV=pickle.load( open('CV.CV', "rb" ) )

In [3]:
def norm_row(inp):
    out=copy.deepcopy(inp)
    for i in range(5,252):
        out[:,i]=(out[:,i]-np.average(inp[:,i-3:i+4]))/np.std(inp[:,i-3:i+4])
    return out

In [4]:
#ampxcos(phase)
import copy
def norm_row(inp):
    out=copy.deepcopy(inp)
    for i in range(5,252):
        out[:,i]=(out[:,i]-np.average(inp[:,i-3:i+4]))/np.std(inp[:,i-3:i+4])
    return out
for fnum in range(70,74):
    #prep figure
    fig=plt.figure(figsize=(8,2))
    axes=fig.subplots(1,2,gridspec_kw={'width_ratios': [4, 1]})
    fig.subplots_adjust(left=0, bottom=0.05, right=0.98, top=0.95, wspace=-0.05, hspace=None)
    axes[0].axis('off')
    axes[1].tick_params(
        bottom=False,         # ticks along the top edge are off
        left=False,         # ticks along the top edge are off
        labelbottom=False, # labels along the bottom edge are off
        labelleft=False) # labels along the bottom edge are off
    # load frame
    fnumstr=str(fnum)
    while len(fnumstr)<3:
        fnumstr='0'+fnumstr
    fname='data/run4_0'+fnumstr+'_fit.lsqfit'
    hybrid_fit=pickle.load( open(fname, "rb" ) )
    # prep frame
    phase=hybrid_fit['phase'][0:79]*np.pi*2
    avg_phase=np.average(phase%np.pi)
    frame=hybrid_fit['amplitude'][0:79]*np.cos(phase-avg_phase)
    frame=norm_row(frame)
    # plot frame
    frame_1d=frame.reshape((-1))
    vmin=np.percentile(frame,1)
    vmax=np.percentile(frame,99)
    cs=axes[0].imshow(frame,vmin=vmin,vmax=vmax,cmap='Greys')
    # plot CV
    axes[1].plot(CV['V'],CV['C']*1E10,color=[0,0,0],lw=0.2)
    CVi=fnum-2
    axes[1].plot(CV['V'][CVi],CV['C'][CVi]*1E10,'o',color=[1,0,0],fillstyle='none')
    # save fig
    fig.savefig('plots/run4_ampxcosphase_'+str(fnum)+'.png',dpi=300)
    plt.close()

In [5]:
#ampxsinphase +diff countourlines
def sg2d(frame, w,o,deriv=0):
    frame2=copy.deepcopy(frame)
    for i in range(256):
        frame2[:,i]=scipy.signal.savgol_filter(frame[:,i], w, o,deriv=deriv)
    return frame2
def remove_max_min(frame,p):
    mi=np.percentile(frame,p)
    ma=np.percentile(frame,100-p)
    frame[frame<mi]=mi
    frame[frame>ma]=ma
    return frame

for fnum in range(70,73):
    #prep figure
    fig=plt.figure(figsize=(8,2))
    axes=fig.subplots(1,2,gridspec_kw={'width_ratios': [4, 1]})
    fig.subplots_adjust(left=0, bottom=0.05, right=0.98, top=0.95, wspace=-0.05, hspace=None)
    axes[0].axis('off')
    axes[1].tick_params(
        bottom=False,         # ticks along the top edge are off
        left=False,         # ticks along the top edge are off
        labelbottom=False, # labels along the bottom edge are off
        labelleft=False) # labels along the bottom edge are off
    # load frame 0
    fnumstr=str(fnum)
    while len(fnumstr)<3:
        fnumstr='0'+fnumstr
    fname='data/run4_0'+fnumstr+'_fit.lsqfit'
    hybrid_fit_0=pickle.load( open(fname, "rb" ))
    # load frame 1
    fnumstrp1=str(fnum+1)
    while len(fnumstrp1)<3:
        fnumstrp1='0'+fnumstrp1
    fnamep1='data/run4_0'+fnumstrp1+'_fit.lsqfit'
    hybrid_fit_1=pickle.load( open(fnamep1, "rb" ))
    # prep frame0
    phase=hybrid_fit_0['phase'][0:79]*np.pi*2
    avg_phase=np.average(phase%np.pi)
    frame_0=hybrid_fit_0['amplitude'][0:79]*np.cos(phase-avg_phase)
    frame_0=norm_row(frame_0)
    # prep frame1
    phase=hybrid_fit_1['phase'][0:79]*np.pi*2
    avg_phase=np.average(phase%np.pi)
    frame_1=hybrid_fit_1['amplitude'][0:79]*np.cos(phase-avg_phase)
    frame_1=norm_row(frame_1)
    # plot frame1
    vmin=np.percentile(frame,1)
    vmax=np.percentile(frame,99)
    cs=axes[0].imshow(frame_1,vmin=vmin,vmax=vmax,cmap='Greys')
    # make contour lines
    frame_0=remove_max_min(frame_0,4)
    frame_1=remove_max_min(frame_1,4)
    diff=frame_1-frame_0
    diff0=diff
    #diff=derX(diff)
    diff-=np.average(diff)
    diff=sg2d(diff,3,2,deriv=1)
    diff=np.abs(diff)
    diff=scipy.ndimage.gaussian_filter(diff,2)
    diff[0]=scipy.ndimage.gaussian_filter(diff[0],2)
    diff[-1]=scipy.ndimage.gaussian_filter(diff[-1],2)
    diff[1]=scipy.ndimage.gaussian_filter(diff[1],2)
    diff[-2]=scipy.ndimage.gaussian_filter(diff[-2],2)
    diff[diff>0.3]=1
    diff[diff<1]=0
    try:
        cs=axes[0].contour(diff,levels=[-0.1,0.2,1.2],linewidths =1.5,colors=[1,0,0])
    except:
        None
    axes[1].plot(CV['V'],CV['C']*1E10,color=[0,0,0],lw=0.2)
    # plot CV
    CVi=fnum-2
    axes[1].plot(CV['V'][CVi+1],CV['C'][CVi+1]*1E10,'o',color=[1,0,0],fillstyle='none')
    # save
    fig.savefig('plots/run4_ampxcosphase_contour'+str(fnum)+'.png',dpi=300)
    plt.close()